In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.colab import drive
drive.mount('/content/drive')  

MessageError: ignored

## Importing the dataset

In [ ]:
#df=pd.read_csv('/content/drive/MyDrive/Example_MS/TwitterSentimentAnalysisUsingCNNDeepLearning/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',header=None)
df=pd.read_csv('/content/drive/MyDrive/enes-example/data.csv',encoding='ISO-8859-1',header=None)
df.head()

FileNotFoundError: ignored

## Feature Engineering

In [ ]:
columns=df.columns
columns

In [ ]:
df.drop([1,2,3,4],axis=1,inplace=True)
df.head()

In [ ]:
df.columns=['sentiment','data']
df.head(100)

In [ ]:
y=df['sentiment']

## Splitting the dataset in train and test split

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test,y_train,y_test=train_test_split(df['data'],y,test_size=0.33,random_state=42)
print('DF Train Shape: ',df_train.shape)
print('DF Test Shape: ',df_test.shape)
print('Y Train Shape: ',y_train.shape)
print('Y Test Shape: ',y_test.shape)


In [ ]:
print(type(df_train),type(df_test),type(y_train),type(y_test))

In [ ]:
lendf_train

## Building deep learn model

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(df_train)
sequence_train=tokenizer.texts_to_sequences(df_train)
sequence_test=tokenizer.texts_to_sequences(df_test)


In [ ]:
type(data_train)

In [ ]:
df_train[0]

In [ ]:
df_train[0]

In [ ]:
word2vec=tokenizer.word_index
V=len(word2vec)
print('dataset has %s number of independent tokens' %V)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
data_train=pad_sequences(sequence_train)
data_train.shape

In [ ]:
T=data_train.shape[1]
data_test=pad_sequences(sequence_test,maxlen=T)
data_test.shape

In [ ]:
print(type(df_train),type(df_test),type(y_train),type(y_test))

**from** tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras.models import Model

In [ ]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding 
from tensorflow.keras.models import Model

In [ ]:
D=20
i=Input((T,))
x=Embedding(V+1,D)(i)
x=Conv1D(32,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(64,3,activation='relu')(x)
x=MaxPooling1D(3)(x)
x=Conv1D(128,3,activation='relu')(x)
x=GlobalMaxPooling1D()(x)
x=Dense(5,activation='softmax')(x)
model=Model(i,x)

model.summary()




In [ ]:

input_layer = Input((7,))
Layer_1 = Dense(70, activation = 'relu') (input_layer)
Layer_2 = Dense(22, activation = 'softmax') (Layer_1)

model2 = Model(input_layer,Layer_2)
model2.summary()


## Training the model

In [ ]:
import datetime
%load_ext tensorboard
from tensorflow.keras.callbacks import TensorBoard
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
data_train

In [ ]:
cnn_senti=model.fit(data_train,y_train,validation_data=(data_test,y_test),epochs=5,batch_size=100,callbacks=[tensorboard_callback])


In [ ]:
y_pred=model.predict(data_test)
y_pred

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
y_pred=np.argmax(y_pred,axis=1)
y_pred

## Scoring 

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

In [ ]:
cm=confusion_matrix(y_test,y_pred)
ax=sns.heatmap(cm,annot=True,cmap='Blues',fmt=' ')
ax.set_title('Confusion Matrix')
ax.set_xlabel('y_test')
ax.set_ylabel('y_pred')


In [ ]:
print(classification_report(y_test,y_pred))